In [4]:
import pandas as pd
import numpy as np
import panel as pn
import panel.widgets as pnw
from panel.pane import Plotly
import param

import hvplot.pandas

# Events Bar Charts

## Video Animation

In [ ]:
video_summer = pn.pane.Video('Resources/summer_race.mp4', loop = False, height=400, width=600)
video_winter = pn.pane.Video('Resources/winter_race.mp4', loop = False,height=400, width=600)

## Athlete Scatter Plot

In [ ]:
import panel as pn
pn.extension('tabulator', 'plotly')
from IPython.display import IFrame

# Obtain the URL of the Tableau chart
tableau_url = "https://public.tableau.com/views/OlympicData-v3-new_bio/Dashboard1?:language=en-US&publish=yes&:display_count=n&:showAppBanner=false&:showVizHome=no&:embed=true&:origin=viz_share_link"

# Create a Panel dashboard and add the Tableau pane
athletes = pn.Tabs(
    ("Tableau Dashboard", pn.pane.HTML(f'<iframe src="{tableau_url}" width="1000" height="1000"></iframe>'))
)
athletes.show()

## Bar and Line Graph

In [ ]:
# Import data for GDP, Population 
df = pd.read_csv('Resources/olympic_data_country_medal_year.csv')

In [ ]:
# add additional data for performance analysis
df['Medal_Per_Capita'] =df['Population']/ df['Y_Score']
df = df.replace([np.inf, -np.inf], 0)
df['Medal_Per_Capita']
minmax = df.copy()

In [ ]:
# Create country list and dropdown
countries = df['Country_Name'].unique().tolist()
select_country = pn.widgets.Select(name='Select', options=countries, 
                                   value='United States')

In [ ]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df

    pn.state.cache['data'] = df.copy()

else: 

    df= pn.state.cache['data']

In [ ]:
# Make the data interactive for pipeline purposes
idf = df.interactive()
idf = idf[(idf.Country_Name == select_country) & (idf.Season == 'Summer')]

In [ ]:
#min max for line gragh
from sklearn.preprocessing import minmax_scale

minmax[['Population', 'GDP', 'GDP_Per_Capita', 'Y_Gold', 'Y_Silver', 'Y_Bronze',
       'Y_Total', 'A_Gold', 'A_Silver', 'A_Bronze', 'A_Total',
       'Y_Capita_Per_Medal', 'A_Capita_Per_Medal', 'Y_Capita_Per_GDP',
       'A_Capita_Per_GDP', 'A_Score', 'Y_Score', 'Y_Score_Per_Capita',
       'A_Score_Per_Capita','Medal_Per_Capita']] = minmax_scale(minmax[['Population', 'GDP', 'GDP_Per_Capita', 'Y_Gold', 'Y_Silver', 'Y_Bronze',
       'Y_Total', 'A_Gold', 'A_Silver', 'A_Bronze', 'A_Total',
       'Y_Capita_Per_Medal', 'A_Capita_Per_Medal', 'Y_Capita_Per_GDP',
       'A_Capita_Per_GDP', 'A_Score', 'Y_Score', 'Y_Score_Per_Capita',
       'A_Score_Per_Capita','Medal_Per_Capita']])

In [ ]:
# Make the data interactive for pipeline purposes, only review Summer
iminmax = minmax.interactive()
iminmax = iminmax[(iminmax.Country_Name == select_country) & (iminmax.Season == 'Summer')]

In [ ]:
# Create the pipeline
ipipeline = iminmax[(iminmax.Country_Name == select_country)]
ipipeline_2 = idf[(idf.Country_Name == select_country)  & (idf.Season == 'Summer')]

In [6]:
# Chart 3 GDP, Population, and GDP Per Capita for Summer using min max scale. Intended to show us Economic health of a country.
# China and India very high population, poor GDP per capita
# Jamaica and Norway, low pop but great GDP
import panel as pn
import plotly.graph_objs as go
import plotly.express as px
pn.extension('plotly')

line_pop = ipipeline.hvplot.line(x='Year', y='Population', xlabel='Year', color = 'red', legend = 'top_left')
line_gdp = ipipeline.hvplot.line(x='Year', y='GDP', xlabel='Year', color = 'blue')
line_cap = ipipeline.hvplot.line(x='Year', y='GDP_Per_Capita', xlabel='Year', ylabel= 'Population, GDP & GDP Per Capita', color = 'green')

hvplot.extension('plotly')
Y_Score = ipipeline.hvplot.bar(stacked=False, 
                                
                                 paper_bgcolor="#F9F9F9",
                                height=1200,
                                width=400,
                                 rot= 60, 
                                 x= 'Year', 
                                 xlabel = "Year", 
                                 y ='Y_Score', 
                                 ylabel = 'Weighted Medal Score')
Chart_3 = line_pop * line_gdp * line_cap * Y_Score

NameError: name 'ipipeline' is not defined

## Bar Chart for Population, GDP and GDP Per Capita

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np

df = pd.read_csv('Resources/olympic_data_country_medal_year.csv')

# Set default year
selected_year = 2020

# Filter data for selected year
def filter_data_by_year(df, selected_year):
    df_filtered = df[df['Year'] == selected_year]
    df_filtered_pop = df_filtered.sort_values(by='Population', ascending=True).tail(25)
    return df_filtered_pop

df_filtered_pop = filter_data_by_year(df, selected_year)

pop_bar = go.Figure()
pop_bar.add_trace(go.Bar(
    x=df_filtered_pop['Population']/5000000,
    y=df_filtered_pop['Country_Name'],
    name='Population',
    marker_color='red',
    orientation='h',
    hovertemplate='%{x:.2f} 5 Million'
))
pop_bar.add_trace(go.Bar(
    x=df_filtered_pop['Y_Score']*-1,
    y=df_filtered_pop['Country_Name'],
    name='Weighted Medal Score',
    marker_color='grey',
    orientation='h',
    opacity=.30
))

years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_pop = filter_data_by_year(df, year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
            "x": [df_filtered_pop['Population']/5000000, df_filtered_pop['Y_Score']*-1],
            "y": [df_filtered_pop['Country_Name']],
            "type": "bar",
            "orientation": "h",
            "marker_color": ['red','grey'],
            "name": ['Population', 'Weighted Medal Score'],
            "hovertemplate": '%{x:.2f} 5 Million'
        },
        {
            "title": f"Country Population and Weighted Medal Score in {year}"
        }]
    ))
# Add dropdown
pop_bar.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1,
        xanchor="right",
        y=1.07,
        yanchor="top"
    )]
)

# Add annotation
pop_bar.update_layout(title=f'Population vs Weighted Medal Score in {selected_year}',
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=.75, y=1.05, yref="paper"),
    ],height=750, width=600, showlegend=False, barmode='relative'
)
import plotly.graph_objs as go
import pandas as pd
import numpy as np

df = pd.read_csv('Resources/olympic_data_country_medal_year.csv')

# Set default year
selected_year = 2020

# Filter data for selected year
def filter_data_by_year(df, selected_year):
    df_filtered = df[df['Year'] == selected_year]
    df_filtered_gdp = df_filtered.sort_values(by='GDP', ascending=True).tail(25)
    return df_filtered_gdp

df_filtered_gdp = filter_data_by_year(df, selected_year)

gdp_bar = go.Figure()
gdp_bar.add_trace(go.Bar(
    x=df_filtered_gdp['GDP']/75000000000,
    y=df_filtered_gdp['Country_Name'],
    name='GDP',
    marker_color='blue',
    orientation='h',
    hovertemplate='%{x:.2f} 75 Billion'
))
gdp_bar.add_trace(go.Bar(
    x=df_filtered_gdp['Y_Score']*-1,
    y=df_filtered_gdp['Country_Name'],
    name='Weighted Medal Score',
    marker_color='grey',
    orientation='h',
    opacity=.30
))

years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_gdp = filter_data_by_year(df, year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
            "x": [df_filtered_gdp['GDP']/75000000000, df_filtered_gdp['Y_Score']*-1],
            "y": [df_filtered_gdp['Country_Name']],
            "type": "bar",
            "orientation": "h",
            "marker_color": ['red','grey'],
            "name": ['Population', 'Weighted Medal Score'],
            "hovertemplate": '%{x:.2f} 5 Million'
        },
        {
            "title": f"GDP vs Weighted Medal Score in {year}"
        }]
    ))
# Add dropdown
gdp_bar.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1,
        xanchor="right",
        y=1.07,
        yanchor="top"
    )]
)

# Add annotation
gdp_bar.update_layout(title=f'GDP vs Weighted Medal Score in {selected_year}',
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=.75, y=1.05, yref="paper"),
    ],height=750, width=600, showlegend=False, barmode='relative'
)
# Filter data for selected year
selected_year = 2020
df_filtered_cap = df.sort_values(by='GDP_Per_Capita', ascending=True).tail(25)
def filter_data_by_year(df, selected_year):
    df_filtered = df[(df['Year'] == selected_year) & (df['Population'] >= 1000000)]
    df_filtered_cap = df_filtered.sort_values(by='GDP_Per_Capita', ascending=True).tail(25)
    return df_filtered_cap

df_filtered_cap = filter_data_by_year(df, selected_year)

cap_bar = go.Figure()
cap_bar.add_trace(go.Bar(
    x=df_filtered_cap['GDP_Per_Capita'],
    y=df_filtered_cap['Country_Name'],
    name='GDP Per Capita',
    marker_color='green',
    orientation='h',
    hovertemplate='%{x:.2f}'
))


years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_cap = filter_data_by_year(df, year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
            "x": [df_filtered_cap['GDP_Per_Capita']],
            "y": [df_filtered_cap['Country_Name']],
            "type": "bar",
            "orientation": "h",
            "marker_color": ['blue','grey'],
            "name": ['GDP Per Capita'],
        },
        {
            "title": f"Country GDP Per Capita in {year}"
        }]
    ))

# Add dropdown
cap_bar.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1,
        xanchor="right",
        y=1.07,
        yanchor="top"
    )]
)

# Add annotation
cap_bar.update_layout(title=f'GDP Per Capita in {selected_year} (Population Size over 1 M)',
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=.75, y=1.05, yref="paper"),
    ],height=750, width=600,showlegend=False
)

## Performance by Capita Scatter Plot

In [ ]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np
df = pd.read_csv('Resources/olympic_data_country_medal_year.csv')
# Create figure
scatter_plot = go.Figure()

# Set default year
selected_year = 2016
df_year = df[df['Year']==selected_year]
# Create two subplots

# Add first chart to subplot 1
scatter_plot = px.scatter(df_year, y="GDP_Per_Capita", x="Y_Capita_Per_Medal", color="Country_Name", hover_name="Country_Name")
scatter_plot.update_traces(marker_size=10, showlegend=True)
scatter_plot.update_layout(scattermode="group", scattergap=0.5)


# Update layout
scatter_plot.update_layout(
    title={
        'text': "Performance by Population Size for 2016",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Medals Per Population",
    yaxis_title="GDP per Capita",
    font=dict(family="Arial", size=12),
    plot_bgcolor="#F9F9F9",
    hovermode="closest",
    margin=dict(l=50, r=50, t=50, b=50),
    paper_bgcolor="#F9F9F9",
    height=500,
    width=800,
)
chart_6 =scatter_plot

## Map

In [9]:
import plotly.express as px
df = pd.read_csv('Resources/olympic_data_country_medal_year.csv')
# GDP MAP
gdp_map = px.choropleth(df, 
                    locations="Country_Code",
                    color="GDP",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Country_Name","GDP","Y_Total", "A_Total"],
                    range_color=[0,5000000000000],
                    color_continuous_scale=px.colors.sequential.Blues)
gdp_map2 = px.scatter_geo(df, locations = "Country_Code",
                    size="A_Total",           
                    hover_data=["Country_Name","GDP","Y_Total", "A_Total"],
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame='Year'
                    )

gdp_map.add_trace(gdp_map2.data[0])
for i, frame in enumerate(gdp_map.frames):
    gdp_map.frames[i].data += (gdp_map2.frames[i].data[0],)

title = {
    'text': 'GDP and Accumulated Medal Wins',
    'y': 0.95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'
}
gdp_map.update_layout(
    title=title,
    height=750,
    width=1000,
    barmode='group',
    showlegend=False
)
pop_map = px.choropleth(df, 
                    locations="Country_Code",
                    color="Population",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Country_Name","Population", "Y_Total","A_Total"],
                    range_color=[15556,1400000000],
                    color_continuous_scale=px.colors.sequential.Reds)
pop_map2 = px.scatter_geo(df, locations = "Country_Code",
                    size="A_Total",           
                    hover_data=["Country_Name","Population", "Y_Total","A_Total"],
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame='Year'
                    )
# POPULATION MAP
pop_map.add_trace(pop_map2.data[0])
for i, frame in enumerate(pop_map.frames):
    pop_map.frames[i].data += (pop_map2.frames[i].data[0],)
title={
    'text': 'Population and Accumulated Medal Wins',
    'y': 0.95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'
}
pop_map.update_layout(
    title=title,
    height=750,
    width=1000,
    barmode='group',
    showlegend=False
)

cap_map = px.choropleth(df, 
                    locations="Country_Code",
                    color="GDP_Per_Capita",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Country_Name","GDP_Per_Capita", "Y_Total","A_Total"],
                    range_color=[0,75000],
                    color_continuous_scale=px.colors.sequential.Greens)
cap_map2 = px.scatter_geo(df, locations = "Country_Code",
                    size="A_Capita_Per_Medal",           
                    hover_data=["Country_Name","GDP_Per_Capita", "Y_Total","A_Total"],
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame='Year',
                    size_max=60
                    )
# POPULATION MAP
cap_map.add_trace(cap_map2.data[0])
for i, frame in enumerate(pop_map.frames):
    cap_map.frames[i].data += (cap_map2.frames[i].data[0],)
title={
    'text': 'GDP Per Capita and Accumulated Medal Wins',
    'y': 0.95,
    'x': 0.5,
    'xanchor': 'center',
    'yanchor': 'top'
}
cap_map.update_layout(
    title=title,
    height=750,
    width=1000,
    barmode='group',
    showlegend=False
)


In [16]:
gdp_map.show()

In [ ]:
import panel as pn
pn.extension()
map_group = pn.widgets.RadioButtonGroup(
    name='Map Button Group', options=['Population', 'GDP', 'GDP Per Capita'], button_type='success')
# Define the map function
@pn.depends(map_group)
def update_map(x):
    if x == 'Population':
        return pn.Column(pop_map, sizing_mode='stretch_both')
    elif x == 'GDP':
        return pn.Column(gdp_map, sizing_mode='stretch_both')
    elif x == 'GDP Per Capita':
        return pn.Column(cap_map, sizing_mode='stretch_both')
    return Plotly(update_map)


In [ ]:
@pn.depends(map_group.param.value)
def update_bar(x):
    if x == 'Population':
        return pn.Column(pop_bar, sizing_mode='stretch_both')
    elif x == 'GDP':
        return pn.Column(gdp_bar, sizing_mode='stretch_both')
    elif x == 'GDP Per Capita':
        return pn.Column(cap_bar, sizing_mode='stretch_both')
    return Plotly(update_bar)

# Predictions

In [ ]:
# Import and transform the data
forest_importances = pd.read_csv('Resources/forest_importances.csv')
forest_importances = forest_importances.rename(columns={'Unnamed: 0': 'Sport'})
forest_importances.head()
fi = forest_importances.transpose()
fi.columns = fi.iloc[0]
fi = fi[1:]
# Create the dropdown menu
sport = fi.columns.tolist()
select_sport2 = pn.widgets.Select(name='Select', options = sport, value = 'Athletics',
    description='Select Sport:')
# cache data to improve dashboard performance
if 'data2' not in pn.state.cache.keys():

    forest_importances

    pn.state.cache['data2'] = forest_importances.copy()

else: 

    forest_importances= pn.state.cache['data2']
iforest = forest_importances.interactive()
#Create the pipeline
hvplot.extension('plotly')
predict = iforest[(iforest.Sport == select_sport2)]
prediction_bar_char = predict.hvplot.bar(width=800, rot=90)

## Creating Dashboard

In [12]:
# Dropdown menu for panel
import panel as pn
from panel.interact import interact
pn.extension()

import hvplot.pandas

import holoviews as hv
from holoviews import opts
hv.extension('plotly')


# Define the Select widget
dropdown = pn.widgets.Select(options=['Seasons', 'Athletes Bio', 'Countries', 'Perfomance by Capita', 'Predictions'])

# Define the page function
@pn.depends(dropdown)
def page(x):
    if Countries:
       return pn.Column(
        pn.Row(map_group),
        pn.Row(
            pn.Column(update_map),
            pn.Column(update_bar)
    ))
    elif x == 'Perfomance by Capita':
        return pn.Row(
       scatter_plot,Chart_3,
            margin=(10, 10),sizing_mode='stretch_both')
    elif x == 'Predictions':
        return pn.Row(
       prediction_bar_char,
            margin=(10, 10),sizing_mode='stretch_both')
# Create the dashboard with the Select widget and the page function
dashboard = pn.Column(
    page,
    sizing_mode='stretch_both'
)

# Display the dashboard
#dashboard.servable()  # or pn.serve(dashboard)

NameError: name 'Countries' is not defined

In [ ]:
template = pn.template.FastListTemplate(
      title='Olympic Medaling Dashboard - Storyboard',
    accent_base_color='#88d8b0',
    header_background='#007bff',
    header_align='center',
    sidebar=[dropdown],
    main=[
        pn.Row(dashboard)
        ],
    )
template.show()

In [11]:
# Original return statement for Seasons in the 'define page function' of Dashboard. Here in case dashboard breaks.
pn.Row(video_summer, video_winter, margin=(10,10), name='Seasons')

NameError: name 'video_summer' is not defined